In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("b-mc2/sql-create-context")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [27]:
import torch

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
model = T5ForConditionalGeneration.from_pretrained('thangved/text2sql').to(device) # type: ignore

In [6]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
import pandas as pd

In [54]:
test_ds = dataset['train'][:100] # type: ignore

In [55]:
test_df = pd.DataFrame(test_ds)

In [56]:
test_df.head()

,answer,question,context
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER)
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA..."
2,"SELECT creation, name, budget_in_billions FROM...","List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam..."
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...
4,SELECT AVG(num_employees) FROM department WHER...,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...


In [57]:
def converter_x(x:dict):
    question = x['question']
    context = x['context']
    
    return f'query for: {question}? </s> tables: {context}'

def converter_x_tensor(x: dict):
    question = x['question']
    context = x['context']
    
    return tokenizer(
        f'query for: {question}? ',
        f'tables: {context}',
        max_length=200,
        padding="max_length",
        truncation=True,
        pad_to_max_length=True,
        add_special_tokens=True,
        return_tensors='pt'
        ).to(device)

def converter_y(x: dict):
    answer = x['answer']
    return answer


X = list(map(converter_x, test_df.iloc)) # type: ignore
X_tensor = list(map(converter_x_tensor, test_df.iloc)) # type: ignore
Y = list(map(converter_y, test_df.iloc)) # type: ignore

In [34]:
import evaluate

2023-11-15 12:29:28.073277: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 12:29:28.147805: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 12:29:28.545409: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 12:29:28.545458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 12:29:28.547259: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [35]:
bleu = evaluate.load('bleu')

In [58]:
answers = []

for x_tensor in X_tensor:
    y_pred = model.generate(input_ids=x_tensor['input_ids'], attention_mask=x_tensor['attention_mask'], max_length=128)
    answer = tokenizer.decode(y_pred.flatten(), skip_special_tokens=True)
    
    print(answer)
    
    answers.append(answer)

SELECT COUNT(*) FROM head WHERE age > 56
SELECT name, born_state, age FROM head ORDER BY age
SELECT creation, name, budget_in_billions FROM department
SELECT MAX(budget_in_billions) FROM department
SELECT AVG(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15
SELECT name FROM head WHERE born_state = 'California'
SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id WHERE T2.born_state = 'Alabama'
SELECT born_state FROM head GROUP BY born_state HAVING COUNT(*) >= 3
SELECT COUNT(*) FROM department GROUP BY creation ORDER BY COUNT(*) DESC LIMIT 1
SELECT T1.name, T1.num_employees FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id WHERE T2.temporary_acting = 'Yes'
SELECT COUNT(DISTINCT temporary_acting) FROM management
SELECT COUNT(*) FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id WHERE T2.department_id = "Supp" AND T1.department_id > T2.department_id
SELE

In [59]:
bleu.compute(predictions=answers, references=Y)

{'bleu': 0.6950776947825641,
 'precisions': [0.8435689455388181,
  0.7533145841703496,
  0.6651318543323567,
  0.5910878112712975],
 'brevity_penalty': 0.9831486212513595,
 'length_ratio': 0.9832890239270794,
 'translation_length': 2589,
 'reference_length': 2633}